## Przygotowanie Środowiska Python dla obliczeń kwatnowych


```bash
python3 -m venv venv
source venv/bin/activate
# Scripts\Activate

pip install qiskit==0.44.1
pip install qiskit[visualization]
# pip install 'qiskit[visualization]'
pip install qiskit_aer==0.12.2
pip install qiskit_machine_learning==0.6.1
pip install qiskit-finance==0.3.4
pip install qiskit-ibmq-provider==0.20.2
pip install qiskit-ignis==0.7.1
```

In [5]:
print(__qiskit_version__)

{'qiskit-terra': '0.25.1', 'qiskit': '0.44.1', 'qiskit-aer': '0.12.2', 'qiskit-ignis': None, 'qiskit-ibmq-provider': '0.20.2', 'qiskit-nature': None, 'qiskit-finance': '0.3.4', 'qiskit-optimization': '0.5.0', 'qiskit-machine-learning': '0.6.1'}


In [6]:
import qiskit.tools.jupyter

%qiskit_version_table

In [ ]:
import qiskit

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

# do przechowywania inforamacji o kubitach przygotowujemy obiekt QuantumRegister
# podajemy ilość kubitów - jako parametr
qreq = QuantumRegister(4)

In [ ]:
# do przechowywania informacji o stanie kubitu po pomiarze potrzebujemy obiektu ClassicalRegister
# przechowujemy w nim już tylko klasyczne bity
creg = ClassicalRegister(4)

In [ ]:
# Tworzenie Quantum Circuit - obiektu reprezentującego obliczenia kwantowe
# składa się z obiektów QuantumRegister i ClassicalRegister
circuit = QuantumCircuit(qreq, creg)

In [ ]:
circuit.draw()

In [ ]:
# zdefiniujmy jeszcze inny rejest kubitów
output = QuantumRegister(1)

In [ ]:
circuit2 = QuantumCircuit(qreq, output, creg)

# jesli nie przewidujemy zadnych pomiarów 
circuit3 = QuantumCircuit(qreq)

In [10]:
from qiskit import Aer

In [11]:
Aer.backends()

[AerSimulator('aer_simulator'),
 AerSimulator('aer_simulator_statevector'),
 AerSimulator('aer_simulator_density_matrix'),
 AerSimulator('aer_simulator_stabilizer'),
 AerSimulator('aer_simulator_matrix_product_state'),
 AerSimulator('aer_simulator_extended_stabilizer'),
 AerSimulator('aer_simulator_unitary'),
 AerSimulator('aer_simulator_superop'),
 QasmSimulator('qasm_simulator'),
 StatevectorSimulator('statevector_simulator'),
 UnitarySimulator('unitary_simulator'),
 PulseSimulator('pulse_simulator')]

## workflow projektów Qiskit

1. Build -  stworzenie obwodu kwantowego dla reprezentacji analizowanego problemu
2. Compile - kompilacja obwodu na symulatorze lub komputerze kwantowym
3. Run - uruchomienie obwodu na symulatorze lub komputerze kwantowym
4. Analyze - analiza wyników

In [ ]:
# Step 1 Import pakietów 

# QuantumCircuit - obiekt reprezentujący obliczenia kwantowe
from qiskit import QuantumCircuit, transpile
# AerSimulator - symulator kwantowy
from qiskit_aer import AerSimulator
# plot_histogram - funkcja do rysowania histogramów
from qiskit.visualization import plot_histogram

In [ ]:

# Step 2 Inicjalizacja zmiennych 
# Create a Quantum Circuit acting on the q register
circuit = QuantumCircuit(2, 2)


$$
\ket{\psi} = \frac{1}{\sqrt{2}}(\ket{00}+\ket{11})
$$

In [ ]:
# Step 3 Dodanie bramek
# Add a H gate on qubit 0
circuit.h(0)

# Add a CX (CNOT) gate on control qubit 0 and target qubit 1
circuit.cx(0, 1)

# Map the quantum measurement to the classical bits
circuit.measure([0, 1], [0, 1])

In [ ]:
# Step 4: Wizualizacja obwodu

circuit.draw("mpl")

In [ ]:
# Step 5 Symulacja obliczeń kwantowych

# Use Aer's AerSimulator
simulator = AerSimulator()
# Compile the circuit for the support instruction set (basis_gates)
# and topology (coupling_map) of the backend
compiled_circuit = transpile(circuit, simulator)

# Execute the circuit on the aer simulator
job = simulator.run(compiled_circuit, shots=1000)

# Grab results from the job
result = job.result()

# Returns counts
counts = result.get_counts(compiled_circuit)
print("\nTotal count for 00 and 11 are:", counts)

In [ ]:
# Step 6: Wizualizacja wyników
# Plot a histogram
plot_histogram(counts)

## Kwantowe obracanie monety

In [7]:
from qiskit import (QuantumCircuit, QuantumRegister, ClassicalRegister, 
                    execute, Aer, __qiskit_version__)

## Uruchomienie kodu na prawdziwym komputerze kwantowym (IBM Q)

Token ibm quantum

In [8]:
# simple circuit
from qiskit import (QuantumCircuit, QuantumRegister, ClassicalRegister, 
                    IBMQ, execute, transpile, Aer, assemble)
from qiskit.tools.monitor import job_monitor

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

qr = QuantumRegister(5)
cr = ClassicalRegister(5)

circuit = QuantumCircuit(qr, cr)
circuit.x(qr[0])
circuit.x(qr[1])
circuit.ccx(qr[0], qr[1], qr[2])
circuit.cx(qr[0], qr[1])
circuit.measure(qr, cr)

In [9]:
 # replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
IBMQ.save_account("TOKEN", overwrite=True) 
provider = IBMQ.load_account()

RequestsApiError: '401 Client Error: Unauthorized for url: https://auth.quantum-computing.ibm.com/api/users/loginWithToken. Login failed., Error code: 3446.'

In [ ]:
# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_quito')

# prepare the circuit for the backend
mapped_circuit = transpile(circuit, backend=backend)
qobj = assemble(mapped_circuit, backend=backend, shots=1024)

# execute the circuit
job = backend.run(qobj)